In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, fbeta_score
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression


In [ ]:
data = pd.read_csv('train.csv')

In [164]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
data['Survived'].sum() # число выживших

342

In [ ]:
data.isna().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [ ]:
print(data.head())

   PassengerId  Survived  Pclass  ...     Fare Cabin  Embarked
0            1         0       3  ...   7.2500   NaN         S
1            2         1       1  ...  71.2833   C85         C
2            3         1       3  ...   7.9250   NaN         S
3            4         1       1  ...  53.1000  C123         S
4            5         0       3  ...   8.0500   NaN         S

[5 rows x 12 columns]


Представляется, что наилучшей метрикой в данной задаче будет F1-score, т.к.ее использование позволит избежать ситуации несбалансированности классов (выживших/невыживших), когда высокая точность может вводить в заблуждение из-за преобладания одного класса, а также позволяет учитывать как точность (сколько из предсказанных положительных примеров действительно положительные), так и полноту (сколько из реальных положительных примеров было предсказано правильно), что важно в контексте предсказания выживаемости.

In [ ]:
data = data.drop(columns=['Name', 'Ticket', 'Cabin', 'Embarked']) # уберем столбцы с не самыми важными данными
data

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,male,22.0,1,0,7.2500
1,2,1,1,female,38.0,1,0,71.2833
2,3,1,3,female,26.0,0,0,7.9250
3,4,1,1,female,35.0,1,0,53.1000
4,5,0,3,male,35.0,0,0,8.0500
...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000
887,888,1,1,female,19.0,0,0,30.0000
888,889,0,3,female,NaN,1,2,23.4500
889,890,1,1,male,26.0,0,0,30.0000


In [ ]:
data['Age'] = data['Age'].fillna(data['Age'].median()) # заполним пропуски средними занчениями


In [ ]:
# Определяем признаки и целевую переменную
X = data.drop(columns=['Survived'])
y = data['Survived']

In [ ]:
# Преобразование категориальных признаков
X['Sex'] = X['Sex'].map({'male': 0, 'female': 1})
X.fillna(0, inplace=True)  # Заполнение остальных пропусков

In [ ]:
print(X)
print(y)

     PassengerId  Pclass  Sex   Age  SibSp  Parch     Fare
0              1       3  0.0  22.0      1      0   7.2500
1              2       1  0.0  38.0      1      0  71.2833
2              3       3  0.0  26.0      0      0   7.9250
3              4       1  0.0  35.0      1      0  53.1000
4              5       3  0.0  35.0      0      0   8.0500
..           ...     ...  ...   ...    ...    ...      ...
886          887       2  0.0  27.0      0      0  13.0000
887          888       1  0.0  19.0      0      0  30.0000
888          889       3  0.0   0.0      1      2  23.4500
889          890       1  0.0  26.0      0      0  30.0000
890          891       3  0.0  32.0      0      0   7.7500

[891 rows x 7 columns]
0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25) # разбиение датасета на тренировочную/тестовую выборки

In [ ]:
train_distribution = y_train.value_counts(normalize=True) # Проверка распределения целевой переменной в обучающем наборе
print("Распределение целевой переменной в обучающем наборе:")
print(train_distribution)

test_distribution = y_test.value_counts(normalize=True) # Проверка распределения целевой переменной в тестовом наборе
print("\nРаспределение целевой переменной в тестовом наборе:")
print(test_distribution)

Распределение целевой переменной в обучающем наборе:
Survived
0    0.613764
1    0.386236
Name: proportion, dtype: float64

Распределение целевой переменной в тестовом наборе:
Survived
0    0.625698
1    0.374302
Name: proportion, dtype: float64


In [ ]:
dummy_clf = DummyClassifier(strategy='most_frequent') # Бейслайн: наиболее частотный класс
dummy_clf.fit(X_train, y_train)

y_pred_dummy = dummy_clf.predict(X_test)

In [ ]:
print("Dummy Classifier Metrics:")
print("Accuracy:", accuracy_score(y_test, y_pred_dummy))
print("Precision:", precision_score(y_test, y_pred_dummy,zero_division=0))
print("Recall:", recall_score(y_test, y_pred_dummy, zero_division=0))
print("F1 Score:", f1_score(y_test, y_pred_dummy, zero_division=0))
print("ROC AUC:", roc_auc_score(y_test, y_pred_dummy))

Dummy Classifier Metrics:
Accuracy: 0.6256983240223464
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
ROC AUC: 0.5


In [ ]:
# логистическая регрессия
log_reg = LogisticRegression(max_iter=200)
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)

print("Logistic Regression Metrics:")
print("Accuracy:", accuracy_score(y_test, y_pred_log_reg))
print("Precision:", precision_score(y_test, y_pred_log_reg, zero_division=0))
print("Recall:", recall_score(y_test, y_pred_log_reg, zero_division=0))
print("F1 Score:", f1_score(y_test, y_pred_log_reg, zero_division=0))
print("ROC AUC:", roc_auc_score(y_test, y_pred_log_reg))

Logistic Regression Metrics:
Accuracy: 0.7150837988826816
Precision: 0.6428571428571429
Recall: 0.5373134328358209
F1 Score: 0.5853658536585366
ROC AUC: 0.6793710021321961


сравнение метрик

In [ ]:
print(f'Precision for Dummy Classificator: {precision_score(y_test, y_pred_dummy, zero_division=0)}')
print(f'Precision for Logistic regression: {precision_score(y_test, y_pred_log_reg, zero_division=0)}')

Precision for Dummy Classificator: 0.0
Precision for Logistic regression: 0.6428571428571429


In [ ]:
print(f'F1-score for Dummy Classificator: {f1_score(y_test, y_pred_dummy)}')
print(f'F1-score for Logistic regression: {f1_score(y_test, y_pred_log_reg)}')

F1-score for Dummy Classificator: 0.0
F1-score for Logistic regression: 0.5853658536585366


In [ ]:
print(f'Fbeta-score for Dummy Classificator (recall bias): {fbeta_score(y_test, y_pred_dummy, beta=2)}') # добавим сдвиг в сторону полноты
print(f'Fbeta-score for Logistic regression (recall bias): {fbeta_score(y_test, y_pred_log_reg, beta=2)}')

Fbeta-score for Dummy Classificator (recall bias): 0.0
Fbeta-score for Logistic regression (recall bias): 0.5555555555555556


In [ ]:
print(f'Fbeta-score for Dummy Classificator (precision bias): {fbeta_score(y_test, y_pred_dummy, beta=0.5)}') # сдвиг в сторону точности
print(f'Fbeta-score for Logistic regression (precision bias): {fbeta_score(y_test, y_pred_log_reg, beta=0.5)}')

Fbeta-score for Dummy Classificator (precision bias): 0.0
Fbeta-score for Logistic regression (precision bias): 0.6185567010309279


In [167]:
# Обучение модели Random Forest
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Предсказание
y_pred_rforest = model.predict(X_test)

# Оценка качества модели
accuracy_model = accuracy_score(y_test, y_pred_rforest)
precision_model = precision_score(y_test, y_pred_rforest)
recall_model = recall_score(y_test, y_pred_rforest)
f1_model = f1_score(y_test, y_pred_rforest)

In [168]:
print("Random Forest Classifier:")
print("Accuracy:", accuracy_score(y_test, y_pred_rforest))
print("Precision:", precision_score(y_test, y_pred_rforest, zero_division=0))
print("Recall:", recall_score(y_test, y_pred_rforest, zero_division=0))
print("F1 Score:", f1_score(y_test, y_pred_rforest, zero_division=0))
print("ROC AUC:", roc_auc_score(y_test, y_pred_rforest))

Random Forest Classifier:
Accuracy: 0.7094972067039106
Precision: 0.6229508196721312
Recall: 0.5671641791044776
F1 Score: 0.59375
ROC AUC: 0.6809035181236673


In [176]:
print(f'F1-score for Logistic regression: {f1_score(y_test, y_pred_rforest)}')

F1-score for Logistic regression: 0.59375


In [178]:
print(f'Fbeta-score for Logistic regression (precision bias): {fbeta_score(y_test, y_pred_rforest, beta=0.5)}')# сдвиг в сторону точности

Fbeta-score for Logistic regression (precision bias): 0.6109324758842444
